In [36]:
# The basics
import numpy as np
import pandas as pd
import bert
import tensorflow_hub as hub
import tensorflow as tf

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Softmax, GRU, Input
from tqdm import tqdm

# utils
import os
import subprocess

In [2]:
parent_relative_path = ".."

In [3]:
colab = 'google.colab' in str(get_ipython())

In [4]:
if colab:
    print("Colab babe")
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)

    os.system('cp "/content/gdrive/My Drive/Colab Notebooks/.kaggle/kaggle.json" /root/.kaggle/kaggle.json')
    subprocess.run(["git", "clone", "https://github.com/codefupanda/customer_interaction_summary.git"])
    os.system('cd customer_interaction_summary && make requirements && make data > logs.logs')
    parent_relative_path = "./customer_interaction_summary"

In [7]:
isear = pd.read_csv(parent_relative_path + '/data/raw/isear.csv', sep='|', error_bad_lines=False, usecols=['Field1', 'SIT', 'EMOT'])

In [8]:
number_of_classes = len(isear.EMOT.unique())

In [9]:
MAX_SEQ_LEN=128
input_word_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,name="input_word_ids")
input_mask = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,name="input_mask")
segment_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,name="segment_ids")

In [10]:
bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=True)



In [11]:
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


In [12]:
def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [28]:
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

In [29]:
FullTokenizer=bert.bert_tokenization.FullTokenizer

vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()

do_lower_case=bert_layer.resolved_object.do_lower_case.numpy()

tokenizer=FullTokenizer(vocab_file,do_lower_case)


In [30]:
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [31]:
def create_single_input(sentence,MAX_LEN):
  
  stokens = tokenizer.tokenize(sentence)
  
  stokens = stokens[:MAX_LEN]
  
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)

  return ids,masks,segments

In [32]:
def create_input_array(sentences):

  input_ids, input_masks, input_segments = [], [], []

  for sentence in tqdm(sentences,position=0, leave=True):
  
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2)

    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [19]:
#tokenizer = Tokenizer(num_words=max_words)
#tokenizer.fit_on_texts(isear['SIT'])
#sequences = tokenizer.texts_to_sequences(isear['SIT'])

In [22]:
#word_index = tokenizer.word_index
#print('Found %s unique tokens.' % len(word_index))

In [23]:
#data = pad_sequences(sequences, maxlen=maxlen, padding='post')

In [52]:
x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
x = tf.keras.layers.Dropout(0.2)(x)
out = tf.keras.layers.Dense(6, activation="sigmoid", name="dense_output")(x)

model = tf.keras.models.Model(
      inputs=[input_word_ids, input_mask, segment_ids], outputs=out)

model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])


In [72]:
train_sentences = isear['SIT'].fillna("CVxTz").values
#list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = isear['EMOT']
inputs=create_input_array(train_sentences)
#model.fit(inputs,train_y,epochs=1,batch_size=32,validation_split=0.2,shuffle=True)
#model.fit(x_train, y_train,epochs=5,batch_size=32,validation_data=(x_test, y_test))


100%|██████████| 7666/7666 [00:03<00:00, 2255.20it/s]


ValueError: in user code:

    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:789 run_step  **
        outputs = model.t

NameError: name 'data' is not defined

In [ ]:
## Model creation time

In [32]:
## Model creation time

In [33]:
model = Sequential()
model.add(Embedding(max_words, output_dim=50, input_length=maxlen))
model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(Flatten())
model.add(Dense(number_of_classes + 1,  activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 50)          500000    
_________________________________________________________________
gru_1 (GRU)                  (None, 1000, 128)         69120     
_________________________________________________________________
flatten_1 (Flatten)          (None, 128000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 1024008   
Total params: 1,593,128
Trainable params: 1,593,128
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(x_train, to_categorical(y_train),
          epochs=5,
          batch_size=32,
          validation_data=(x_test, to_categorical(y_test)))

Epoch 1/5
174/180 [============================>.] - ETA: 6s - loss: 0.2867 - accuracy: 0.9046

KeyboardInterrupt: 

In [16]:
y_pred = model.predict_classes(x_test)
y_pred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([3, 7, 4, ..., 4, 1, 3])

In [ ]:
confusion_matrix(y_test, y_pred)